## 0.&nbsp;Instalación Spark

In [1]:
# Instalar SDK Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Descargar Spark 3.2.3
!wget -q https://archive.apache.org/dist/spark/spark-3.2.3/spark-3.2.3-bin-hadoop3.2.tgz

# Descomprimir el archivo descargado de Spark
!tar xf spark-3.2.3-bin-hadoop3.2.tgz

# Establecer las variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.3-bin-hadoop3.2"

# Instalar la librería findspark
!pip install -q findspark

# Instalar pyspark
!pip install -q pyspark==3.2.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 15.8 MB/s eta 0:00:00


## 1.&nbsp;Spark Session

In [2]:
# Find Spark
import findspark
findspark.init()

# Spark Session
from pyspark.sql import SparkSession

# SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

## 2.&nbsp;Ejercicios

In [3]:
"""
Ejercicio 1: Cree un RDD llamado lenguajes que contenga los siguientes
lenguajes de programación: Python, R, C, Scala, Rugby y SQL.
"""
lenguajes_programacion = sc.parallelize(["Python", "R", "C", "Scala", "Rugby", "SQL"])
lenguajes_programacion.collect()

['Python', 'R', 'C', 'Scala', 'Rugby', 'SQL']

In [4]:
"""
a. Obtenga un nuevo RDD a partir del RDD lenguajes donde todos
los lenguajes de programación estén en mayúsculas.
"""
lenguajes_programacion_upper = lenguajes_programacion.map(lambda x: x.upper())
lenguajes_programacion_upper.collect()

['PYTHON', 'R', 'C', 'SCALA', 'RUGBY', 'SQL']

In [5]:
"""
b. Obtenga un nuevo RDD a partir del RDD lenguajes donde todos los lenguajes
de programación estén en minúsculas.
""""
lenguajes_programacion_lower = lenguajes_programacion.map(lambda x: x.lower())
lenguajes_programacion_lower.collect()

['python', 'r', 'c', 'scala', 'rugby', 'sql']

In [6]:
"""
c. Cree un nuevo RDD que solo contenga aquellos lenguajes de programación
que comiencen con la letra R.
"""
lenguajes_programacion_startwith_r = lenguajes_programacion.filter(lambda x: x.startswith("R"))
lenguajes_programacion_startwith_r.collect()

['R', 'Rugby']

In [23]:
"""
Ejercicio 2: Cree un RDD llamado pares que contenga los números
pares existentes en el intervalo [20;30]
"""
pares = sc.parallelize([x for x in range(20, 31) if x % 2 == 0])
pares.collect()

[20, 22, 24, 26, 28, 30]

In [24]:
"""
a. Cree el RDD llamado sqrt, este debe contener la raíz cuadrada de
los elementos que componen el RDD pares.
"""
sqrt = pares.map(lambda x: x ** (1/2))
sqrt.collect()

[4.47213595499958,
 4.69041575982343,
 4.898979485566356,
 5.0990195135927845,
 5.291502622129181,
 5.477225575051661]

In [25]:
"""
Obtenga una lista compuesta por los números pares en el intervalo [20;30]
y sus respectivas raíces cuadradas. Un ejemplo del resultado deseado para
el intervalo [50;60] sería la lista [50, 7.0710678118654755, 52, 7.211102550927978, 54, 7.3484692283495345, 56, 7.483314773547883, 58, 7.615773105863909, 60, 7.745966692414834].
"""
pares_sqrt = pares.flatMap(lambda x: (x, x ** (1/2)))
pares_sqrt.collect()

[20,
 4.47213595499958,
 22,
 4.69041575982343,
 24,
 4.898979485566356,
 26,
 5.0990195135927845,
 28,
 5.291502622129181,
 30,
 5.477225575051661]

In [26]:
"""
c. Eleve el número de particiones del RDD sqrt a 20.
"""
sqrt_20 = sqrt.repartition(20)
sqrt_20.getNumPartitions()

20

In [27]:
"""
d. Si tuviera que disminuir el número de particiones luego de haberlo
establecido en 20, ¿qué función utilizaría para hacer más eficiente su código?
"""
sqrt_10 = sqrt_20.coalesce(10)
sqrt_10.getNumPartitions()

10

In [28]:
"""
Ejercicio 3: Cree un RDD del tipo clave valor a partir de los datos adjuntos
como recurso a esta lección. Tenga en cuenta que deberá procesar el RDD leído
para obtener el resultado solicitado. Supongamos que el RDD resultante de tipo
clave valor refleja las transacciones realizadas por número de cuentas.
Obtenga el monto total por cada cuenta.
"""
transacciones_rdd = sc.textFile("./transacciones")
transacciones_rdd.collect()

['(1001, 52.3)',
 '(1005, 20.8)',
 '(1001, 10.1)',
 '(1004, 52.7)',
 '(1005, 20.7)',
 '(1002, 85.3)',
 '(1004, 20.9)']

In [29]:
transacciones_tuples_rdd = transacciones_rdd.map(lambda line: tuple(line.replace("(", "").replace(")", "").split(", ")))
transacciones_tuples_rdd.collect()

[('1001', '52.3'),
 ('1005', '20.8'),
 ('1001', '10.1'),
 ('1004', '52.7'),
 ('1005', '20.7'),
 ('1002', '85.3'),
 ('1004', '20.9')]

In [30]:
transacciones_key_value_rdd = transacciones_tuples_rdd.map(lambda x: (x[0], float(x[1])))
transacciones_key_value_rdd.collect()

[('1001', 52.3),
 ('1005', 20.8),
 ('1001', 10.1),
 ('1004', 52.7),
 ('1005', 20.7),
 ('1002', 85.3),
 ('1004', 20.9)]

In [31]:
transacciones_reduced_rdd = transacciones_key_value_rdd.reduceByKey(lambda x, y: x + y)
transacciones_reduced_rdd.collect()

[('1002', 85.3), ('1001', 62.4), ('1005', 41.5), ('1004', 73.6)]